# day 08

finding connected components; programmed live in class

In [1]:
from util import file_to_list
from math import sqrt

In [2]:
def read_boxes(raw):
    boxes = []
    for line in raw:
        str_boxes = line.split(',')
        int_boxes = [int(s) for s in str_boxes]
        boxes.append(int_boxes)
    return boxes

In [3]:
def dist(a, b):
    x1, y1, z1 = a
    x2, y2, z2 = b
    distance = (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2
    return sqrt(distance)

In [4]:
assert dist([0,0,0], [1,1,1]) == sqrt(3)

In [5]:
def pairwise(boxes):
    N = len(boxes)
    distances = []
    for i in range(N-1):
        for j in range(i+1, N):
            d = dist(boxes[i], boxes[j])
            distances.append([d, i, j])
    distances.sort()
    return distances

In [6]:
def merge(circuits, merge_from, merge_to):
    merged = []
    for c in circuits:
        if c == merge_from:
            merged.append(merge_to)
        else:
            merged.append(c)
    return merged

In [7]:
def form_circuits(boxes, distances, top):
    N = len(boxes)
    circuits = [-1] * N
    current_circuit = 0

    for _, i, j in distances[:top]:
        # none have a circuit
        if circuits[i] == -1 and circuits[j] == -1:
            circuits[i] = current_circuit
            circuits[j] = current_circuit
            current_circuit += 1
        # one of them has a circuit
        elif -1 in [circuits[i], circuits[j]]:
            m = max(circuits[i], circuits[j])
            circuits[i] = m
            circuits[j] = m
        # both already have a circuit
        else:
            circuits = merge(circuits, circuits[i], circuits[j])
    return circuits

In [8]:
def circuit_size(circuits):
    freq = []
    for v in set(circuits):
        if v < 0:
            continue
        freq.append(circuits.count(v))
    freq.sort(reverse=True)
    return freq

def topN(freq, n=3):
    result = 1
    for v in freq[:n]:
        result *= v
    return result

In [9]:
assert topN([100]) == 100

In [10]:
def part1(path, top):
    raw = file_to_list(path)
    boxes = read_boxes(raw)
    distances = pairwise(boxes)
    circuits = form_circuits(boxes, distances, top)
    freq = circuit_size(circuits)
    return topN(freq)

In [11]:
part1('./data/day08_test.txt', 10)

40

In [12]:
part1('./data/day08.txt', 1000)

57564

## part 2:

we now go all the way until we have a single component graph

In [13]:
def endgame(circuits, N):
    freq = circuit_size(circuits)
    # all boxes are assigned to one circuit
    all_boxes_accounted_for = sum(freq) == N
    # one circuit to rule them all
    all_in_one = len(freq) == 1
    # print(sum(freq), len(freq))
    return all_boxes_accounted_for and all_in_one

In [14]:
def form_circuits_pro(boxes, distances):
    N = len(boxes)
    circuits = [-1] * N
    current_circuit = 0

    for _, i, j in distances:
        # none have a circuit
        if circuits[i] == -1 and circuits[j] == -1:
            circuits[i] = current_circuit
            circuits[j] = current_circuit
            current_circuit += 1
        # one of them has a circuit
        elif -1 in [circuits[i], circuits[j]]:
            m = max(circuits[i], circuits[j])
            circuits[i] = m
            circuits[j] = m
        # both already have a circuit
        else:
            # are we at endgame?
            circuits = merge(circuits, circuits[i], circuits[j])

        # check if we put everything in one circuit
        # and that circuit contains all boxes
        if endgame(circuits, N):
            return boxes[i][0] * boxes[j][0]
    return None

In [15]:
def part2(path):
    raw = file_to_list(path)
    boxes = read_boxes(raw)
    distances = pairwise(boxes)
    x_product = form_circuits_pro(boxes, distances)
    return x_product

In [16]:
part2('./data/day08_test.txt')

25272

In [17]:
part2('./data/day08.txt')

133296744